<a href="https://colab.research.google.com/github/GaganKumar2375/Palm-Tree-Detection-using-YOLOv8-and-R-CNN/blob/main/YOLOv8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# YOLOv8 Code

In [ ]:
!pip install ultralytics


In [ ]:

!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="A2wycrFsDr1E3JqnNRlT")
project = rf.workspace("nur-byq0f").project("palm-detection-4qh3m")
version = project.version(2)
dataset = version.download("yolov8")


In [ ]:
from ultralytics import YOLO

# Load the pre-trained YOLOv8 model
model = YOLO("yolov8n.pt")  # Options: yolov8n.pt, yolov8s.pt, yolov8m.pt, etc.

# Train the model
model.train(
    data="/content/Palm-Detection-2/data.yaml",       # Path to the YAML configuration file
    epochs=10,              # Number of training epochs
    imgsz=640,              # Image size for training
    batch=16,               # Batch size
    device=0                # Use GPU (0) or CPU ('cpu')
)


In [ ]:
# Evaluate the trained model
metrics = model.val()
print(metrics)  # This prints mAP, precision, recall, etc.


In [ ]:
import matplotlib.pyplot as plt

# Run validation and get real YOLOv8 metrics
metrics = model.val()  # This evaluates the model and returns a dictionary of metrics

# Extract relevant metrics
yolov8_metrics = {
    "AP @[IoU=0.50:0.95] (all)": metrics.box.map,       # mAP@0.5:0.95
    "AP @[IoU=0.50] (all)": metrics.box.map50,          # mAP@0.5
}

# Filter out None values (if any)
valid_metrics = {k: v for k, v in yolov8_metrics.items() if v is not None}

# Plot the metrics
labels = list(valid_metrics.keys())
values = list(valid_metrics.values())

plt.figure(figsize=(12, 8))
plt.barh(labels, values, color="skyblue")
plt.xlabel("Metric Value")
plt.title("YOLOv8 Evaluation Metrics (from model.val())")
plt.grid(axis="x", linestyle="--", alpha=0.7)
plt.tight_layout()
plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import os

# Path to the YOLOv8 runs
detect_path = 'runs/detect'

# Filter only folders that contain 'results.csv'
valid_runs = []
for run in os.listdir(detect_path):
    results_csv = os.path.join(detect_path, run, 'results.csv')
    if os.path.isfile(results_csv):
        valid_runs.append((run, os.path.getmtime(results_csv)))

# Sort by modification time
if not valid_runs:
    raise FileNotFoundError("No valid run folders with 'results.csv' found.")

latest_run = sorted(valid_runs, key=lambda x: x[1])[-1][0]
results_path = os.path.join(detect_path, latest_run)
metrics_file = os.path.join(results_path, 'results.csv')

print(f"Using results from: {results_path}")

# Load the CSV
df = pd.read_csv(metrics_file)
df.columns = df.columns.str.strip()

# Training Losses
plt.figure(figsize=(12, 6))
plt.plot(df['train/box_loss'], label='Box Loss')
plt.plot(df['train/cls_loss'], label='Class Loss')
plt.plot(df['train/dfl_loss'], label='DFL Loss')
plt.title("YOLOv8 Training Loss Curves")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
import os
import glob
from IPython.display import display
from PIL import Image as PILImage
import matplotlib.pyplot as plt

# Get prediction image paths
val_preds = glob.glob(os.path.join(results_path, 'val_batch*_pred.jpg'))

# Display each image nicely using matplotlib
for i, img_path in enumerate(val_preds[:3]):
    img = PILImage.open(img_path)
    plt.figure(figsize=(8, 8))
    plt.imshow(img)
    plt.axis("off")
    plt.title(f"YOLO Prediction {i + 1}")
    plt.show()
